In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from keras.models import model_from_json
from datetime import datetime
from keras.applications import inception_v3 as inc_net
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D#, GlobalMaxPooling2D 
from keras.models import load_model
from keras.callbacksTest import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
import os

Using TensorFlow backend.


In [3]:
num_classes = 128
img_size = 299
my_batch_size = 128
my_steps_per_epoch = 850
my_val_steps = 18
my_epochs_basic = 3
my_epochs_retrain = 12
is_shuffle = True
write_batch_per = True

In [4]:
# idk needs a loot of improvement, but just for testing
base_model = inc_net.InceptionV3(weights='imagenet', include_top=False)
print("InceptionV3 loaded")
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#freezing all layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range = 0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set= train_datagen.flow_from_directory('./data/mushrooms_filled/train/',
                                                target_size = (img_size, img_size),
                                                batch_size=my_batch_size,
                                                class_mode='categorical')
val_set = test_datagen.flow_from_directory('./data/mushrooms_filled/test/',
                                           target_size= (img_size, img_size),
                                           batch_size=my_batch_size,
                                           class_mode='categorical')

Found 107874 images belonging to 128 classes.
Found 2232 images belonging to 128 classes.


In [6]:
time_of_test = str(datetime.now().strftime("%d-%m-%y %H:%M"))
dir_file_name= time_of_test+'_Epochs'+str(my_epochs_basic)+'_Steps'+str(my_steps_per_epoch)+'_ImageSize'+str(img_size)+'_BatchSize'+str(my_batch_size)
file_name = 'mushrooms_pretrain_'+dir_file_name+'.h5'
path = './logs/pretrain/'+dir_file_name
if not os.path.exists(path):
    os.mkdir(path)
model.fit_generator(training_set,
                    steps_per_epoch=my_steps_per_epoch,
                    epochs=my_epochs_basic,
                    validation_data=val_set,
                    validation_steps=50,
                    shuffle=is_shuffle,
                    callbacks=[TensorBoard(log_dir=path, write_batch_performance=write_batch_per)])

Epoch 1/3
850/850 [==============================] - 4455s 5s/step - loss: 2.9137 - acc: 0.3123 - val_loss: 3.7003 - val_acc: 0.2020
Epoch 2/3
850/850 [==============================] - 4434s 5s/step - loss: 1.8478 - acc: 0.5226 - val_loss: 3.8245 - val_acc: 0.2113
Epoch 3/3
850/850 [==============================] - 4368s 5s/step - loss: 1.4674 - acc: 0.6078 - val_loss: 3.9684 - val_acc: 0.2230


In [7]:
# SAVE THE CURRENT MODEL
model.save('./trainedModels/pretrain/'+file_name)  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk as> "+file_name)

Saved model to disk as> mushrooms_pretrain_25-06-18 15:18_Epochs3_Steps850_ImageSize299_BatchSize128.h5


In [5]:
# LOAD THE CURRENT MODEL
#print()
file_name= './trainedModels/pretrain/mushrooms_pretrain_25-06-18 15:18_Epochs3_Steps850_ImageSize299_BatchSize128.h5'
model = load_model(file_name)

print('Model loaded> '+file_name)

Model loaded> ./trainedModels/pretrain/mushrooms_pretrain_25-06-18 15:18_Epochs3_Steps850_ImageSize299_BatchSize128.h5


In [6]:
#tbCallBack_finetune = TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True), loss='categorical_crossentropy', metrics=['accuracy'] )

In [9]:
time_of_real = str(datetime.now().strftime("%d-%m-%y %H:%M"))
dir_file_name= time_of_real+'__Epochs'+str(my_epochs_retrain)+'_Steps'+str(my_steps_per_epoch)+'_ImageSize'+str(img_size)+'_BatchSize'+str(my_batch_size)
file_name = dir_file_name+'.h5'
path = './logs/realtrain/'+dir_file_name

model.load_weights('./ModelCheckpoints/26-06-18 16:15__weights.11-2.52.hdf5')

if not os.path.exists(path):
    os.mkdir(path)

model.fit_generator(training_set,
                    steps_per_epoch=10,
                    epochs=1, 
                    validation_data=val_set,
                    validation_steps=16,
                    shuffle=is_shuffle,
                    #initial_epoch=11,
                    callbacks=[TensorBoard(log_dir=path,
                                           write_batch_performance=write_batch_per),
                              ModelCheckpoint('./ModelCheckpoints/'
                                              +time_of_real
                                              +'__weights.{epoch:02d}-val_acc{val_acc:.2f}--train_acc{acc:.2f}.hdf5',
                                              monitor='val_loss',
                                              verbose=1,
                                              save_best_only=False,
                                              save_weights_only=False,
                                              mode='auto',
                                              period=1)
                              ]
                   )

Epoch 1/1
1/1 [==============================] - 37s 37s/step - loss: 0.0715 - acc: 0.9844 - val_loss: 2.7709 - val_acc: 0.5176

Epoch 00001: saving model to ./ModelCheckpoints/26-06-18 23:26__weights.01-val_acc0.52--train_acc0.98.hdf5


In [8]:
## SAVE THE MODEL
print('try to save model')
model.save('./trainedModels/retrain/'+file_name)  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk as> "+file_name)

try to save model
Saved model to disk asmushrooms_realtrain_26-06-18 21:40__Epochs12_Steps850_ImageSize299_BatchSize128.h5
